In [1]:
import pandas as pd
import numpy as np
import sklearn.ensemble as ek
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import TransformerMixin
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import lime
import lime.lime_tabular
from __future__ import print_function

C:\Users\Neha Gupta\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataset=pd.read_csv(r"C:\Users\Neha Gupta\Desktop\ndd.txt",header=None)
X = dataset.drop([3,2,1,41],axis=1).values
y = dataset[41].values

In [3]:
X

array([[0.00e+00, 4.91e+02, 0.00e+00, ..., 5.00e-02, 0.00e+00, 2.00e+01],
       [0.00e+00, 1.46e+02, 0.00e+00, ..., 0.00e+00, 0.00e+00, 1.50e+01],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 1.90e+01],
       ...,
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 1.00e+00, 1.00e+00, 2.10e+01],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 2.00e+01],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 1.80e+01]])

In [4]:
attacks=dataset[41].unique()

In [5]:
attacks

array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy'], dtype=object)

In [6]:
extratrees = ek.ExtraTreesClassifier().fit(X,y)
model = SelectFromModel(extratrees, prefit=True)
X_new = model.transform(X)
nbfeatures = X_new.shape[1]

# Logistic regression

In [7]:
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, auc ,roc_curve

In [8]:
X_train, X_validation, Y_train, Y_validation = cross_validation.train_test_split(X_new, y ,test_size=0.2)

In [9]:
model_logreg = LogisticRegression()
model_logreg.fit(X_train, Y_train)
accuracy_score(Y_validation, model_logreg.predict(X_validation))

0.9519745981345505

In [11]:
print(classification_report(Y_validation,model_logreg.predict(X_validation)))

                 precision    recall  f1-score   support

           back       0.91      0.65      0.75        31
buffer_overflow       0.00      0.00      0.00         1
   guess_passwd       1.00      0.33      0.50         3
           imap       0.00      0.00      0.00         1
        ipsweep       0.81      0.90      0.85       152
       multihop       0.00      0.00      0.00         1
        neptune       0.99      1.00      1.00      1630
           nmap       0.62      0.41      0.50        61
         normal       0.96      0.99      0.97      2707
            pod       0.00      0.00      0.00        10
      portsweep       0.99      0.89      0.94       111
        rootkit       0.00      0.00      0.00         1
          satan       0.88      0.69      0.78       143
          smurf       0.78      0.78      0.78       115
       teardrop       0.19      0.08      0.12        36
    warezclient       0.71      0.77      0.74        35
    warezmaster       0.00    

C:\Users\Neha Gupta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
cm = confusion_matrix(Y_validation, model_logreg.predict(X_validation))
print('confusion matrix\n %s' % cm)


confusion matrix
 [[  20    0    0    0    0    0    0    0   10    0    0    0    0    0
     0    1    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    1    0]
 [   2    0    1    0    0    0    0    0    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0  137    0    0    3   10    0    0    0    0    0
     0    2    0]
 [   0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0    0]
 [   0    0    0    0    0    0 1629    0    0    0    0    0    0    0
     1    0    0]
 [   0    0    0    0   27    0    0   25    5    0    0    0    0    0
     4    0    0]
 [   0    0    0    0    5    0    3    3 2667    0    0    0    9   13
     2    5    0]
 [   0    0    0    0    1    0    0    4    3    0    0    0    0    0
     2    0    0]
 [   0    0    0    0    0    0    6    0    6    0   99    0    0    0
     0    

# Random forest

In [12]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, Y_train)
accuracy_score(Y_validation, model_rf.predict(X_validation))

0.9940464377852749

In [13]:
print(classification_report(Y_validation,model_rf.predict(X_validation)))

                 precision    recall  f1-score   support

           back       1.00      0.97      0.98        31
buffer_overflow       0.00      0.00      0.00         1
      ftp_write       0.00      0.00      0.00         0
   guess_passwd       1.00      1.00      1.00         3
           imap       0.00      0.00      0.00         1
        ipsweep       0.97      1.00      0.99       152
       multihop       0.00      0.00      0.00         1
        neptune       1.00      1.00      1.00      1630
           nmap       0.97      0.95      0.96        61
         normal       0.99      1.00      1.00      2707
            pod       1.00      0.60      0.75        10
      portsweep       1.00      0.98      0.99       111
        rootkit       1.00      1.00      1.00         1
          satan       0.99      0.97      0.98       143
          smurf       0.99      0.97      0.98       115
       teardrop       0.94      0.94      0.94        36
    warezclient       0.97    

C:\Users\Neha Gupta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Neha Gupta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
cm = confusion_matrix(Y_validation, model_rf.predict(X_validation))
print('confusion matrix\n %s' % cm)

confusion matrix
 [[  30    0    1    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    1    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    3    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    1    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0  152    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    1    0    0    0    0    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    0    0 1629    0    1    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    1    0    0   58    2    0    0    0    0
     0    0    0    0]
 [   0    0    0    0    0    2    0    0    2 2702    0    0    0    0
     0    0    1    0]
 [   0    0    0    0    0    0 

# Lime

In [17]:
predict_fn_logreg = lambda x: model_logreg.predict_proba(x).astype(float)
predict_fn_rf = lambda x: model_rf.predict_proba(x).astype(float)

In [23]:
features = []
index = np.argsort(extratrees.feature_importances_)[::-1][:nbfeatures]

In [24]:
index.astype(np.float64)

array([29., 21., 25., 35., 34., 32., 22., 24., 33., 38., 36., 31., 37.,
        8., 19.])

In [25]:
train_int = [30., 21., 25.,  8., 35., 28., 29., 32., 38., 23., 19., 33., 31.]
features_name_int = list(train_int)
feature_names = sum([features_name_int],[])
print(feature_names)

[30.0, 21.0, 25.0, 8.0, 35.0, 28.0, 29.0, 32.0, 38.0, 23.0, 19.0, 33.0, 31.0]


In [31]:

explainer = lime.lime_tabular.LimeTabularExplainer(X_train,feature_names = index,class_names=attacks,
                                                   categorical_features='none', 
                                                   categorical_names='none', kernel_width=3)

In [37]:
X_validation

array([[  0.  ,   1.  ,   1.  , ...,   0.  ,   0.  ,  18.  ],
       [  0.  , 140.  ,   1.  , ...,   0.  ,   0.  ,  20.  ],
       [  1.  ,   3.  ,   0.33, ...,   0.  ,   0.  ,  20.  ],
       ...,
       [  0.  , 199.  ,   0.  , ...,   0.  ,   0.  ,  18.  ],
       [  0.  ,   2.  ,   0.  , ...,   1.  ,   1.  ,  15.  ],
       [  1.  ,   5.  ,   0.  , ...,   0.  ,   0.  ,  21.  ]])

In [38]:
observation_1 = 2
exp = explainer.explain_instance(X_validation[observation_1], predict_fn_logreg, num_features=13)

In [40]:
exp.as_list()

[('22 <= 11.00', 9.385663761824782e-05),
 ('29 <= 0.00', -7.678146613005252e-05),
 ('14.00 < 21 <= 143.00', -7.454951288486105e-05),
 ('18.00 < 19 <= 20.00', -4.090812028671124e-05),
 ('37 <= 0.00', 3.1439689028465586e-05),
 ('33 <= 0.00', -2.9359820231710495e-05),
 ('0.00 < 25 <= 1.00', -2.3432308755230074e-05),
 ('8 <= 0.00', 1.8175064375673938e-05),
 ('0.02 < 24 <= 0.07', 1.4348108483306865e-05),
 ('32 <= 0.09', 1.066167706847058e-05),
 ('34 <= 0.00', 4.42502142044697e-06),
 ('38 <= 0.00', -3.2212661531490455e-06),
 ('0.00 < 31 <= 1.00', -2.9781328126476587e-06)]

In [41]:
exp = explainer.explain_instance(X_validation[observation_1], predict_fn_rf, num_features=13)
exp.as_list()

[('29 <= 0.00', -0.00014927787876533707),
 ('33 <= 0.00', -0.00011485126284801456),
 ('0.02 < 24 <= 0.07', -8.568893366597969e-05),
 ('0.00 < 25 <= 1.00', -7.783517983539767e-05),
 ('38 <= 0.00', 7.759255829172838e-05),
 ('18.00 < 19 <= 20.00', -7.555974016132e-05),
 ('32 <= 0.09', -6.523059911227696e-05),
 ('8 <= 0.00', 5.890874769285609e-05),
 ('34 <= 0.00', 5.87595691982064e-05),
 ('37 <= 0.00', -5.724411982867059e-05),
 ('14.00 < 21 <= 143.00', 1.6070920621750752e-05),
 ('22 <= 11.00', 1.2622077497084125e-05),
 ('0.00 < 35 <= 1.00', 9.8663413677419e-06)]

In [42]:
print(Y_validation[observation_1])

neptune


In [51]:
preds=model_logreg.predict(X_validation)

In [52]:
preds

array(['nmap', 'neptune', 'normal', ..., 'normal', 'portsweep', 'normal'],
      dtype=object)

In [53]:
preds[observation_1]

'neptune'

In [55]:
observation_2 = 30
exp = explainer.explain_instance(X_validation[observation_2], predict_fn_logreg, num_features=13)
exp.as_list()

[('21 <= 2.00', 0.0002007681158303084),
 ('19 <= 18.00', 0.00018725056322408027),
 ('0.00 < 29 <= 1.00', 8.562146375533552e-05),
 ('24 <= 0.00', 6.0500755276210206e-05),
 ('33 > 0.06', 5.5483904662837e-05),
 ('0.00 < 31 <= 1.00', 5.3071845118027663e-05),
 ('36 <= 0.00', 4.683399720454729e-05),
 ('11.00 < 22 <= 62.00', 4.230266331984456e-05),
 ('8 <= 0.00', 3.716948800192401e-05),
 ('37 <= 0.00', 2.3977325198240126e-05),
 ('35 <= 0.00', -2.0496158916881253e-05),
 ('38 > 0.02', -1.2843302225378013e-05),
 ('25 <= 0.00', 1.0774707981901261e-05)]

In [56]:
exp = explainer.explain_instance(X_validation[observation_2], predict_fn_rf, num_features=13)
exp.as_list()

[('19 <= 18.00', 0.000665436147189512),
 ('24 <= 0.00', 0.000487863456337097),
 ('0.00 < 29 <= 1.00', 0.00046836363848367805),
 ('33 > 0.06', 0.0003908686849164199),
 ('11.00 < 22 <= 62.00', 0.00032221077252170635),
 ('21 <= 2.00', 0.00030276509311954325),
 ('25 <= 0.00', 0.00025924587703289955),
 ('37 <= 0.00', 0.0002391255110394568),
 ('8 <= 0.00', 0.00023201221041378088),
 ('35 <= 0.00', -0.00017191343857222012),
 ('36 <= 0.00', -0.00016763924013021554),
 ('38 > 0.02', -0.00013993420512776875),
 ('0.00 < 31 <= 1.00', 5.61817822560019e-05)]